## 📖 Pipeline de Processamento de Dados Climáticos
- Objetivo: Este notebook tem como objetivo carregar dados brutos de clima em formato JSON, processá-los, limpar sua estrutura e transformá-los em um DataFrame tabular pronto para análise.

### 1. Configuração e Importação de Bibliotecas

In [1]:
import pandas as pd
import json
import pytz
from datetime import datetime, timedelta


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### 2. Carregamento dos Dados (Extração)

In [3]:
path = '../data/raw/recife_weather=2025-06-12_16-16-56.json'

with open(path, 'r', encoding='utf-8') as f:
    data_json = json.load(f)

### 3. Normalização e Junção Inicial do JSON
- Os dados brutos foram extraídos de um JSON estruturado, contendo informações principais e uma subestrutura chamada `weather`. Utilizei o  `pd.json_normalize()` para "achatar" essas estruturas em DataFrames separados (`df_inicial` e `df_weather`) e, em seguida, combinei ambos com `pd.concat()`.

In [4]:
df_inicial = pd.json_normalize(data_json)
df_weather = pd.json_normalize(data_json['weather'])


df_inicial = df_inicial.drop('weather', axis=1)
df = pd.concat([df_weather, df_inicial], axis=1)

df

,id,main,description,icon,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,main.sea_level,main.grnd_level,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,802,Clouds,scattered clouds,03d,stations,10000,1749755324,-10800,3390760,Recife,200,-34.8811,-8.0539,28.02,31.26,28.02,28.02,1013,74,1013,1010,6.17,170,40,1,8426,BR,1749716984,1749758956


### 3.1 Analisando informações do DF

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1 non-null      int64  
 1   main             1 non-null      object 
 2   description      1 non-null      object 
 3   icon             1 non-null      object 
 4   base             1 non-null      object 
 5   visibility       1 non-null      int64  
 6   dt               1 non-null      int64  
 7   timezone         1 non-null      int64  
 8   id               1 non-null      int64  
 9   name             1 non-null      object 
 10  cod              1 non-null      int64  
 11  coord.lon        1 non-null      float64
 12  coord.lat        1 non-null      float64
 13  main.temp        1 non-null      float64
 14  main.feels_like  1 non-null      float64
 15  main.temp_min    1 non-null      float64
 16  main.temp_max    1 non-null      float64
 17  main.pressure    1 n

In [7]:
df.shape

(1, 29)

In [8]:
df.columns

Index(['id', 'main', 'description', 'icon', 'base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod', 'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like', 'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity', 'main.sea_level', 'main.grnd_level', 'wind.speed', 'wind.deg', 'clouds.all', 'sys.type', 'sys.id', 'sys.country', 'sys.sunrise', 'sys.sunset'], dtype='object')

### 4. Renomeação de Colunas para Evitar Conflitos

In [9]:
df = df.rename(columns={
    'id': 'weather_id',
    'main': 'weather_main',
    'description': 'weather_description'
})

### 4.1 Remoção de Colunas Desnecessárias

In [10]:
df = df.drop(['icon', 'base', 'main.grnd_level', 'main.sea_level', 'sys.type', 'sys.id', 'cod', 'weather_id'], axis=1)

### 5. Renomeação final das colunas

In [ ]:
columns_rename = {
    'weather_main': 'weather_condition',
    'dt': 'measurement_datetime',
    'visibility': 'visibility_km',
    'name': 'city',
    'coord.lon': 'longitude',
    'coord.lat': 'latitude',
    'sys.country': 'country_code',
    'main.temp': 'temperature',
    'main.feels_like': 'feels_like_temp',
    'main.temp_min': 'temp_min',
    'main.temp_max': 'temp_max',
    'main.pressure': 'pressure',
    'main.humidity': 'humidity',
    'wind.speed': 'wind_speed',
    'wind.deg': 'wind_direction_degrees',
    'clouds.all': 'cloudiness_percent',
    'sys.sunrise': 'sunrise_datetime',
    'sys.sunset': 'sunset_datetime'
}

df = df.rename(columns=columns_rename)

### 6. Arredondamento das Colunas de Temperatura

In [12]:
temp_columns = ['feels_like_temp', 'temp_max', 'temp_min'], 'temperature'

for column in temp_columns:
    df[column] = df[column].round(1)


###  7. Conversão do Fuso Horário (timezone)
- O valor da coluna `timezone` (em segundos) foi convertido para um `pd.Timedelta`, representando o deslocamento de horário em relação ao UTC. Isso foi usado para ajustar as datas posteriormente.

In [13]:
# timezone = -10800
timezone = df['timezone'].iloc[0]
offset = pd.Timedelta(seconds=timezone)
offset

Timedelta('-1 days +21:00:00')

### 8. Conversão de timestamps para datetime

In [ ]:
date_columns = ['measurement_datetime', 'sunrise_datetime', 'sunset_datetime']

for column in date_columns:
    df[column] = pd.to_datetime(df[column], unit='s')


### 8.1 Ajuste do Fuso Horário nos timestamps
- Adicionei o `offset` de timezone às colunas de data, convertendo os horários para o fuso horário local do local medido.

In [ ]:
date_columns = ['measurement_datetime', 'sunrise_datetime', 'sunset_datetime']
for column in date_columns:
    df[column] =  df[column] + offset

### 8.2. Criação da coluna `record_datetime`
- Foi criada a coluna `record_datetime` copiando os valores de `measurement_timestamp`, para facilitar futuras manipulações ou análises sem alterar a coluna original.

In [ ]:
df['record_timestamp'] = df['measurement_datetime']

###  10. Extração do horário (sem a data) 

In [18]:
date_columns = ['measurement_timestamp', 'sunrise_timestamp', 'sunset_timestamp']

for column in date_columns:
    df[column] = df[column].dt.time

### 11. Conversão da visibilidade para km
- A coluna `visibility_km` originalmente está em metros. Realizamos a conversão para quilômetros dividindo por 1000, o que facilita a leitura e interpretação dos dados.

In [19]:
df['visibility_km'] = df['visibility_km'] / 1000

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   weather_condition       1 non-null      object        
 1   weather_description     1 non-null      object        
 2   visibility_km           1 non-null      float64       
 3   measurement_timestamp   1 non-null      object        
 4   timezone                1 non-null      int64         
 5   city                    1 non-null      object        
 6   longitude               1 non-null      float64       
 7   latitude                1 non-null      float64       
 8   temperature             1 non-null      float64       
 9   feels_like_temp         1 non-null      float64       
 10  temp_min                1 non-null      float64       
 11  temp_max                1 non-null      float64       
 12  pressure                1 non-null      int64         

In [24]:
df

,weather_condition,weather_description,visibility_km,measurement_timestamp,timezone,city,longitude,latitude,temperature,feels_like_temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_direction_degrees,cloudiness_percent,country_code,sunrise_timestamp,sunset_timestamp,record_datetime,record_year
0,Clouds,scattered clouds,10.0,16:08:44,-10800,Recife,-34.8811,-8.0539,28.0,31.3,28.0,28.0,1013,74,6.17,170,40,BR,05:29:44,17:09:16,2025-06-12 16:08:44,2025-06-12
